# Bibliotecas utilizadas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import roc_auc_score
import xgboost
import shap
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.max_columns=None

In [ ]:
DATA_PATH = '../data/'

In [ ]:
df = pd.read_csv(DATA_PATH+'dados_manipulados_v2_edicao_1_clean.csv', sep=';')

# Funções utilizadas

In [ ]:
def analise_cat_1(df,var,label = '',fl_ordena=0, num = False, q = 0, q2 = 0, y = '', ajst_taxa = False,tx_real_evento=0.027):
    '''
    Analise bivariada
    '''  
    if ajst_taxa == True:
        qty_amostras_real = df.loc[df[y]==1].shape[0] / tx_real_evento
       
        if df.shape[0] < qty_amostras_real:
            n = int(qty_amostras_real - df.shape[0])
            df_sel = (df[[var, y]].loc[df[y]==0]).iloc[list(np.random.choice(df.loc[df[y]==0].shape[0], n))]
            df = pd.concat([df[[var, y]],df_sel])
   
    if label == '':
        label = var
    df_ = df.copy()
    if num:
        if q == 0:
            if q2 == 0:
                df_[var+'_cat'] = pd.qcut(df_[var],q= [0.0,0.2,0.5,0.8,1.0],retbins=True,duplicates='drop')[0].cat.add_categories('missing')
                var= var+'_cat'
            else:
                df_[var+'_cat'] = pd.qcut(df_[var],q= q2,retbins=True,duplicates='drop')[0].cat.add_categories('missing')
                var= var+'_cat'
        else :
            df_[var+'_cat'] = pd.cut(df_[var],bins=q).cat.add_categories('missing')
            var= var+'_cat'
    print('\n')
    print('Análise da variável {}'.format(var))
    print('# valores distintos {}'.format(df_[var].nunique()))
    try:
        df_[var] = df_[var].fillna('missing')
    except :
        df_[var] = df_[var].cat.add_categories('missing').fillna('missing')
   
    total_maus = df_[y].sum()
    total_bons = df_.shape[0] - df_[y].sum()

    def tx(x):
        return sum(x)*100/sum(1 -x +x)
    def pct(x):
        return sum(1-x+x)*1.0/ (df_.shape[0])
    def pct_maus(x):
        return (sum(x)/total_maus )
    def rr(x):
        return (sum(x)/total_maus )/(sum(1-x)/total_bons)
    def WoE(x):
        return np.log(rr(x))
    def IV(x):
        return -(sum(1-x)/total_bons - sum(x)/total_maus)*WoE(x)
    s = df_.groupby(var).agg({y: [np.size,pct,pct_maus, tx,rr,WoE,IV]})[y]
    def color_negative_green(val):
        color = 'green' if val < 0 else 'black'
        return 'color: %s' % color
    print('IV : {:.3f}'.format(s.IV.sum()))
    #print('Beta : {:.3f}'.format(dic_beta[var]))
    t = s.style.applymap(color_negative_green)
    display(pd.DataFrame(df_[var].value_counts()).join(pd.DataFrame(df_[var].value_counts(normalize= True)*100), lsuffix = 'k').rename(columns = {var+'k': '#', var: '%'}))
    display(t)
    by_var = df_.groupby(var).agg({y:[np.size, np.mean]})[y]
    if fl_ordena ==1 :
        by_var =  by_var.sort_values(by = 'mean')
    Y1 = by_var['size']
    Y2 =by_var['mean']
    Y_mean = np.ones(shape=(len(Y1.index)))* df_[y].mean()
    index =  np.arange(len(Y1.index))
    #with plt.style.context('my_custom_style'):
    if True:
        plt.figure(figsize=(14,6))
        plt.bar(index,Y1,alpha = 0.3, color= 'gray')
        plt.grid(False)
        plt.xticks(rotation = 30         if var[:2] not in ( 'fl', 'cd')  else 0)
        plt.ylabel('# registros')
        plt.twinx()
        plt.gca().set_xticks(index)
        plt.gca().set_xticklabels([Y1.index[i] for i in index], rotation = 40)
        plt.plot(index,Y_mean,label=  'tx. média evento')
        plt.plot(index,Y2,marker = 'o',label=  'tx. evento')
        plt.gca().set_yticklabels([ ' {:.2f}%'.format( i*100) for i in plt.gca().get_yticks()])
        plt.grid(False)
        plt.title('Bivariada  {}'.format(label))
        plt.ylabel('tx. evento')
        plt.xlabel(label)
        if var[:2] in ( 'fl', 'cd'):
            plt.legend(loc= 9,bbox_to_anchor=(0.5, -0.1))
        else:
            plt.legend(loc= 9,bbox_to_anchor=(0.5, -0.35))
        plt.show()
        print(s.IV.sum())
        return  (var , s.IV.sum(), s)
        print('\n')

def call_analise_list(df, label,l, num=True):
    for col in l:
   
        analise_cat_1(df,col ,y=label, num=num, q2=10  )
    
def create_woe(df, coluna, mapa):
    return mapa.loc[df[coluna]]['WoE']

# Limpeza dos dados 

In [ ]:
df.head() 

In [ ]:
df.shape

In [ ]:
df_filtro = df.loc[~df.var_faturamento.isna()]

In [ ]:
df_filtro.shape

In [ ]:
df_filtro.head() 

In [ ]:
df_filtro.columns.to_list()

In [ ]:
col_to_anl = ['estado_de_origem',
              
              #'custos_materias_prima',
              #'custos_com_pessoal',
              #'custos_com_aluguel',
              #'custos_com_impostos',
              #'custos_emprestimos_dividas',
              'numero_pessoas_ocupadas',
              
              'acao_expandiu_vendas_online',
              'acao_ampliou_presenca_digital',
              'acao_expandiu_meios_entrega',
              
              'acao_reducao_horario_trabalho',
              'acao_revezamento_equipe',
              
              'acao_ferias_funcionarios',
              'acao_demitiu_funcionarios',
              
              'acao_fechou_temporariarmente_empresa',
              'acao_fechou_permanentemente_empresa',
              
              'porte_da_empresa',
              'setor_atividade',
              'tempo_atividade',
              'ramo_atuacao',
              
              'media_faturamento',
              'idade',
              'escolaridade'
             ]

In [ ]:
for val in col_to_anl:
    print('-------------------------------------------------------------------------')
    print('')
    print(f'Variável: {val}')
    display(df_filtro[val].value_counts(normalize=True))
    print('')
    print('-------------------------------------------------------------------------')

In [ ]:
df_filtro.head()

In [ ]:
X = ['estado_de_origem',             
     'numero_pessoas_ocupadas',         
     
     'acao_expandiu_vendas_online',
     'acao_ampliou_presenca_digital',
     'acao_expandiu_meios_entrega',
              
     'acao_reducao_horario_trabalho',
     'acao_revezamento_equipe',
              
     'acao_ferias_funcionarios',
     'acao_demitiu_funcionarios',
              
     'acao_fechou_temporariarmente_empresa',
     'acao_fechou_permanentemente_empresa',
              
     'porte_da_empresa',
     'setor_atividade',
     'tempo_atividade',
     'ramo_atuacao',
              
     'media_faturamento',
     'idade',
     'escolaridade'
             ]
Y = ['var_faturamento'] # variavel dependente

In [ ]:
vars_to_trans = [
     'acao_expandiu_vendas_online',
     'acao_ampliou_presenca_digital',
     'acao_expandiu_meios_entrega',
              
     'acao_reducao_horario_trabalho',
     'acao_revezamento_equipe',
              
     'acao_ferias_funcionarios',
     'acao_demitiu_funcionarios',
              
     'acao_fechou_temporariarmente_empresa',
     'acao_fechou_permanentemente_empresa',


    ]

In [ ]:
for col in vars_to_trans:
    df_filtro.loc[~df_filtro[col].isnull(), col] = 1
    df_filtro[col].fillna(0, inplace=True)

In [ ]:
df_filtro.numero_pessoas_ocupadas.fillna(0, inplace=True)
df_filtro.numero_pessoas_ocupadas = \
df_filtro.numero_pessoas_ocupadas.astype(int)

In [ ]:
df_filtro.head() 

In [ ]:
df_filtro.respondent_id.nunique()

In [ ]:
df_filtro.shape

In [ ]:
df = df_filtro

# Análise de Variáveis

## Variação de Faturamento 

In [ ]:
fig, ax = plt.subplots(figsize=(10,8));
sns.histplot(df_filtro.var_faturamento);
plt.show()

In [ ]:
df_filtro.var_faturamento.describe(percentiles=[.2, .3,.4 ,.5, .7,.8, .85,.9, .95, .97])

## Número de Pessoas Ocupadas

In [ ]:
df_filtro.numero_pessoas_ocupadas.fillna(0, inplace=True)

In [ ]:
df_filtro.numero_pessoas_ocupadas.isna().sum() # verificando se existe valores nulos

In [ ]:
fig, ax = plt.subplots(figsize=(10,8));
sns.histplot(df_filtro.numero_pessoas_ocupadas, bins=50);
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8));
sns.histplot(df_filtro.loc[df_filtro.numero_pessoas_ocupadas<=40, 'numero_pessoas_ocupadas'], bins=50);
plt.show()

In [ ]:
df_filtro.numero_pessoas_ocupadas.describe(percentiles=[.2, .3, .5, .7, .9, .95])

In [ ]:
df_filtro.loc[df_filtro.numero_pessoas_ocupadas<=40, 'numero_pessoas_ocupadas']\
.describe(percentiles=[.2, .3, .5, .7, .9, .95])


In [ ]:
sns.jointplot(data=df_filtro.loc[df_filtro.numero_pessoas_ocupadas<=40], 
              x='numero_pessoas_ocupadas', 
              y='var_faturamento', 
              kind="reg", 
              height=10);

## Modo de Funcionamento das Empresas

In [ ]:
for col in [
     'acao_fechou_temporariarmente_empresa',
    ]:
    display(df_filtro.groupby(col).agg({'var_faturamento':'mean'}))
    df_filtro.groupby(col).agg({'var_faturamento':'mean'}).plot.bar()
    plt.show()

## Ações feitas - Vendas da empresa

In [ ]:
for col in [
     'acao_expandiu_vendas_online',
     'acao_ampliou_presenca_digital',
     'acao_expandiu_meios_entrega',
    ]:
    
    display(df_filtro.groupby(col).agg({'var_faturamento':'mean'}))
    display(df_filtro[col].value_counts(normalize=True))
    df_filtro.groupby(col).agg({'var_faturamento':'mean'}).plot.bar()
    plt.show()

## Ações feitas - Funcionários

In [ ]:
for col in [
     'acao_reducao_horario_trabalho',
     'acao_revezamento_equipe',
              
     'acao_ferias_funcionarios',
     'acao_demitiu_funcionarios',
    ]:
    
    display(df_filtro.groupby(col).agg({'var_faturamento':'mean'}))
    display(df_filtro[col].value_counts(normalize=True))
    df_filtro.groupby(col).agg({'var_faturamento':'mean'}).plot.bar()
    plt.show()

## Estado de Origem

In [ ]:
display(df_filtro.groupby('estado_de_origem').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('estado_de_origem').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Porte da Empresa

In [ ]:
df_filtro.porte_da_empresa.isna().sum()

In [ ]:
df_filtro.loc[df_filtro.porte_da_empresa.isna(), 'var_faturamento'].mean()

In [ ]:
df_filtro.porte_da_empresa.fillna('ME', inplace=True)

In [ ]:
display(df_filtro.groupby('porte_da_empresa').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('porte_da_empresa').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Setor de Atividade da Empresa

In [ ]:
df_filtro.setor_atividade.isna().sum()/df_filtro.shape[0]

In [ ]:
display(df_filtro.groupby('setor_atividade').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('setor_atividade').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Tempo de Atividade

In [ ]:
df_filtro.tempo_atividade.isna().sum()

In [ ]:
display(df_filtro.groupby('tempo_atividade').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('tempo_atividade').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Ramo de Atuação da Empresa

In [ ]:
df_filtro.ramo_atuacao.isna().sum()

In [ ]:
df_filtro.ramo_atuacao_outros.isna().sum()

In [ ]:
display(df_filtro.groupby('ramo_atuacao').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('ramo_atuacao').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Faturamento Médio Mensal da Empresa

In [ ]:
df_filtro.media_faturamento.isna().sum()

In [ ]:
display(df_filtro.groupby('media_faturamento').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('media_faturamento').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Idade do Empresário

In [ ]:
df_filtro.idade.isna().sum()

In [ ]:
display(df_filtro.groupby('idade').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('idade').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

## Escolaridade do Empresário

In [ ]:
df_filtro.escolaridade.isna().sum()

In [ ]:
display(df_filtro.groupby('escolaridade').agg({'var_faturamento':'mean'}).sort_values(by='var_faturamento', ascending=False))
df_filtro.groupby('escolaridade').agg({'var_faturamento':'mean'}).plot.bar()
plt.show()

# Variáveis para modelagem

In [ ]:
X = ['numero_pessoas_ocupadas',
     'acao_fechou_temporariarmente_empresa',
     'acao_expandiu_vendas_online',
     'acao_ampliou_presenca_digital',
     'acao_expandiu_meios_entrega',
     'estado_de_origem',
     'porte_da_empresa',
     'ramo_atuacao',
     'media_faturamento',
     'idade'
     ]

# Definição da Variável Resposta

Explicacao da definição do que é uma empresa "boa" (target = 1)

In [ ]:
#df.loc[df.var_faturamento >= -40, 'target'] = 1
df.loc[df.var_faturamento >= 0, 'target'] = 1
df.target.fillna(0, inplace=True)

# Análise Bivariada

Analise das variaveis x o target (empresas boas)

## Idade

In [ ]:
analise_cat_1(df=df, var='idade' , y='target', num=False, );

## Média de faturamento mensal

Essa variavel não está sentido, era esperado que empresas com um faturamento mensal maior tivessem uma taxa maior de empresas boas, isso não é verificado aqui

In [ ]:
analise_cat_1(df=df, var='media_faturamento' , y='target', num=False, );

In [ ]:

df.loc[df.media_faturamento=='Até 6 mil por mês', 'media_faturamento_mes'] = '1 - Até 6 mil por mês'
df.loc[ (df.media_faturamento=='De 7 a 15 mil por mês')\
       | (df.media_faturamento=='De 16 a 30 mil por mês')\
       | (df.media_faturamento=='De 31 a 60 mil por mês'), 'media_faturamento_mes'] = '2 - de 6 a 60 mil por mês'
df.loc[(df.media_faturamento=='De 60 a 100 mil por mês')\
       | (df.media_faturamento=='Prefiro não responder'), 'media_faturamento_mes'] = '3 - de 60 a 100 mil por mês'
df.loc[(df.media_faturamento=='De 100 mil a 200 mil')\
       | (df.media_faturamento=='Acima de 200 mil'), 'media_faturamento_mes'] = '4 - Acima de 100 mil por mês'
df.loc[(df.media_faturamento=='Não sei/não quero informar'), 'media_faturamento_mes'] = '5 - Não sabe'



In [ ]:
_, _, s_faturamento =  analise_cat_1(df=df, var='media_faturamento_mes' , y='target', num=False, );

In [ ]:
df['faturamento_mensal_woe'] = df.apply(create_woe, axis=1, args=('media_faturamento_mes', s_faturamento))

In [ ]:
analise_cat_1(df=df, var='faturamento_mensal_woe' , y='target', num=True, q2=20);

## Ramo de atuação

In [ ]:
_, _, s_ramo_atuacao = analise_cat_1(df=df, var='ramo_atuacao' , y='target', num=False, );

In [ ]:
s_ramo_atuacao.sort_values(by='WoE')

In [ ]:
df['ramo_atuacao_woe'] = df.apply(create_woe, axis=1, args=('ramo_atuacao', s_ramo_atuacao))

In [ ]:
df['ramo_atuacao_woe'].unique()

In [ ]:
df.loc[df['ramo_atuacao_woe'] == -np.inf, 'ramo_atuacao_woe'] = -100

In [ ]:
analise_cat_1(df=df, var='ramo_atuacao_woe' , y='target', num=True, );

## Estado de Origem

In [ ]:
_, _, s_estado = analise_cat_1(df=df, var='estado_de_origem' , y='target', num=False, );

In [ ]:
df['estado_origem_woe'] = df.apply(create_woe, axis=1, args=('estado_de_origem', s_estado))

In [ ]:
df.loc[df['estado_origem_woe'] == -np.inf, 'estado_origem_woe'] = -1

In [ ]:
analise_cat_1(df=df, var='estado_origem_woe' , y='target', num=True, q2=5);

## Ações feitas - Vendas da empresa

In [ ]:
colunas = [     'acao_expandiu_vendas_online',
     'acao_ampliou_presenca_digital',
     'acao_expandiu_meios_entrega',
]

In [ ]:
call_analise_list(df, label='target',l=colunas, num=False)

In [ ]:
df.loc[df.acao_expandiu_vendas_online==1, 'medidas_expansao_vendas'] = 1
df.loc[df.acao_ampliou_presenca_digital==1, 'medidas_expansao_vendas'] = 1
df.loc[df.acao_expandiu_meios_entrega==1, 'medidas_expansao_vendas'] = 1
df.medidas_expansao_vendas.fillna(0, inplace=True)

In [ ]:
analise_cat_1(df=df, var='medidas_expansao_vendas' , y='target', num=False);

## Funcionamento da empresa

In [ ]:
colunas = [  'acao_fechou_temporariarmente_empresa',
]

In [ ]:
call_analise_list(df, label='target',l=colunas, num=False)

In [ ]:
df.loc[df.acao_fechou_temporariarmente_empresa==1, 'nao_mudou_funcionamento_empresa_na_crise'] = 0
df.loc[df.acao_fechou_temporariarmente_empresa==0, 'nao_mudou_funcionamento_empresa_na_crise'] = 1

In [ ]:
analise_cat_1(df=df, var='nao_mudou_funcionamento_empresa_na_crise' , y='target', num=False);

## Número de pessoas ocupadas

In [ ]:
analise_cat_1(df=df, var='numero_pessoas_ocupadas' , y='target', num=True, q2=10);

## Porte de empresa

In [ ]:
analise_cat_1(df=df, var='porte_da_empresa' , y='target', num=False, q2=0);

# Correlação entre as variáveis

In [ ]:
COLUMNS_SEL = [
               'faturamento_mensal_woe'
               ,'ramo_atuacao_woe'
               ,'estado_origem_woe'
               ,'medidas_expansao_vendas'
               ,'nao_mudou_funcionamento_empresa_na_crise'
               ,'numero_pessoas_ocupadas'
               ,'porte_da_empresa'
               ,'var_faturamento'
              ]

In [ ]:
df[COLUMNS_SEL].head()

In [ ]:
df_tratado = pd.get_dummies(df[COLUMNS_SEL])

In [ ]:
df_tratado.head()

In [ ]:
df_tratado.columns.tolist()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_tratado.corr(), annot=True);
plt.show()

In [ ]:
VALS_TO_DROP =['porte_da_empresa_MEI'
]

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_tratado.drop(VALS_TO_DROP, axis=1).corr(), annot=True);
plt.show()

# Modelagem

## Linear Regression

In [ ]:
X = df_tratado.drop(VALS_TO_DROP+['var_faturamento'], axis=1)
y = df_tratado['var_faturamento']

In [ ]:
X.info()

In [ ]:
df_tratado['var_faturamento'].describe()

In [ ]:
df_tratado['var_faturamento'].isna().sum()

In [ ]:
df.head()

In [ ]:
df['faturamento_mensal'] = df['media_faturamento_mes']
df.loc[df.nao_mudou_funcionamento_empresa_na_crise==1, 'funcionamento_na_crise'] = 'nao_mudou_funcionamento_empresa_na_crise'
df['funcionamento_na_crise'] = df['funcionamento_na_crise'].fillna('Alterou modo de funcionamento')
df['vender_por_ferramentas_digitais_pre_crise'] = np.nan
df['qtdade_demissao_funcionarios_ultimo_mes_covid'] = np.nan

df.loc[(df['acao_reducao_horario_trabalho']==1) |
        (df['acao_revezamento_equipe']==1) |
        (df['acao_ferias_funcionarios']==1),
        'adotou_medidas_com_funcionarios' ] = 1
df['adotou_medidas_com_funcionarios'] = df['adotou_medidas_com_funcionarios'].fillna(0)
df['buscou_emprestimo_crise'] = np.nan

In [ ]:
COLUNAS_EXPORT = [
    'respondent_id', #
    'medidas_expansao_vendas', #
    'numero_pessoas_ocupadas',#
    'funcionamento_na_crise',#
    'adotou_medidas_com_funcionarios', 
    'qtdade_demissao_funcionarios_ultimo_mes_covid',#
    'ramo_atuacao', #
    'porte_da_empresa',#
    'estado_de_origem',#
    'faturamento_mensal',#
    'buscou_emprestimo_crise',
    'vender_por_ferramentas_digitais_pre_crise',#
    'var_faturamento'#
]

In [ ]:
df[COLUNAS_EXPORT].head()

In [ ]:
df[COLUNAS_EXPORT].to_csv('../data/20230524analise_variaveis_pesquisa_1.csv')

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [ ]:
model = LinearRegression().fit(X_std, y)

In [ ]:
model.score(X_std, y)

In [ ]:
model.coef_

In [ ]:
coefs = pd.DataFrame(
    model.coef_,
    columns=['Coefficients'], index=X.columns.tolist()
)

In [ ]:
df_coef = coefs.reindex(coefs['Coefficients'].abs().sort_values(ascending=False).index)

In [ ]:
df_coef.reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="Coefficients", y="index", data=df_coef.reset_index(),
            label="coefs", palette=["seagreen" if x>=0 else 'r' for x in df_coef["Coefficients"]])
plt.show()